# Introduction

We are going to train a convolutional neural network (cnn) to recognize facial emotions. Then we use the output of the emotions to play a game in a browser which is like guitarhero. Instead of pressing the matching button at the right time, you can use your face and make the same emotion as needed. Our program then checks wheter you did a good job (same emotion) or if you weren't that clear with your facial emotion!


## Dataset

As a dataset, we use videos from 23 different actors who sing and say a sentence in eight different emotions. 
Our old approach was the following: We will seperate the videos into each emotion and then take several screenshots from the video.
Our new approach is to use a face landmark recognition api too feed our model with our data to train it on different emotions.


## Credits


Credits:
"[The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS)](https://zenodo.org/record/1188976)" by Livingstone & Russo is licensed under [CC BY-NA-SC 4.0.](https://creativecommons.org/licenses/by-nc-sa/4.0/)


---

In [1]:
import keras
keras.__version__

D:\Programme\Anaconda3\envs\compx\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


'2.2.4'

## Preparing the data

First of all we inport os, shutil and glob. glob is used to match path names.
We are going to copy every video into one folder


In [6]:
import os, shutil, glob
import numpy as np

In [11]:
with open("D:/Arbeit/semester5/comppx18/Projekt/Datasets/KaggleEmotion/fer2013/fer2013.csv") as f:
    content = f.readlines()

    lines = np.array(content)

    num_of_instances = lines.size
print("number of instances: ",num_of_instances)

['emotion,pixels,Usage\n'
 '0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121 119 115 110 98 91 84 84 90 99 110 126 143 153 158 171 169 172 169 165 129 110 113 107 95 79 66 62 56 57 61 52 43 41 65 61 58 57 56 69 75 70 65 56 54 105 146 154 151 151 155 155 150 147 147 148 152 158 164 172 177 182 186 189 188 190 188 180 167 116 95 103 97 77 72 62 55 58 54 56 52 44 50 43 54 64 63 71 68 64 52 66 119 156 161 164 163 164 167 168 170 174 175 176 178 179 183 187 190 195 197 198 197 198 195 191 190 145 86 100 90 65 57 60 54 51 41 49 56 47 38 44 63 55 46 52 54 55 83 138 157 158 165 168 172 171 173 176 179 179 180 182 185 187 189 189 192 197 200 199 196 198 200 198 197 177 91 87 96 58 58 59 51 42 37 41 47 45 37 35 36 30 41 47 59 94 141 159 161 161 164 170 171 172 176 178 179 182 183 183 187 189 192 192 194 195 200 200 199 199 200 201 197 193 111 71 108 69 55 61 51 42 43 56 54 44 24 29 31 45 61 72 100 136 150 159 163 162 163 170 172 171 174 177 177 180 187 186 187 189 192 192 194 195 196 197 199 200

In [255]:
x_train, y_train, x_test, y_test = [], [], [], []
for i in range(1,num_of_instances):
    try:
        emotion, img, usage = lines[i].split(",")

        val = img.split(" ")
        pixels = np.array(val, 'float32')
        emotion = keras.utils.to_categorical(emotion, 7)
        if 'Training' in usage:
            y_train.append(emotion)
            x_train.append(pixels)
        elif 'PublicTest' in usage:
            y_test.append(emotion)
            x_test.append(pixels)
    except:
        print("", end="")

In [260]:
y_train = np.array(y_train)
x_train = np.array(x_train)
y_test = np.array(y_test)
x_test = np.array(x_test)
x_train = x_train.reshape((len(x_train),48,48)+(1,))
x_test = x_test.reshape((len(x_test),48,48)+(1,))

In [257]:
#np.concatenate((np.ones((len(x_train),1), dtype=np.int), x_train.reshape(len(x_train),1)), axis=1)
#print(x_train.shape)
#x_train = np.insert(x_train, 0, values=30, axis=1) 
#print(x_train)
#print(x_train.shape)
print(x_train.shape)
#x_train = x_train.reshape((len(x_train), len(x_train[0])))
#x_train = x_train.astype('float32') / 255

(28709, 48, 48, 1)


In [258]:
from keras import layers
from keras import models
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import Dense
from keras.layers import Dropout


model = models.Sequential()
 
#1st convolution layer
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))
 
#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))
 
#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))
 
model.add(layers.Flatten())
 
#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
 
model.add(Dense(7, activation='softmax'))

In [259]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)
batch_size = 128
gen = ImageDataGenerator()
train_generator = gen.flow(x_train, y_train, batch_size=batch_size)
 
model.compile(loss='categorical_crossentropy'
, optimizer=keras.optimizers.Adam()
, metrics=['accuracy']
)
model.fit_generator(train_generator, steps_per_epoch=batch_size, epochs=1)

Epoch 1/1
 32/128 [======>.......................] - ETA: 33:40 - loss: 13.5400 - acc: 0.1343

KeyboardInterrupt: 

In [ ]:
model.save("faceEmotion1.h5")

In [ ]:
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss:', train_score[0])
print('Train accuracy:', 100*train_score[1])
 
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', test_score[0])
print('Test accuracy:', 100*test_score[1])

In [ ]:
'''from sklearn.metrics import classification_report, confusion_matrix
pred_list = []; actual_list = []
for i in predictions:
pred_list.append(np.argmax(i))
for i in y_test:
actual_list.append(np.argmax(i))
confusion_matrix(actual_list, pred_list)
'''

In [ ]:
def emotion_analysis(emotions):
    objects = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
    y_pos = np.arange(len(objects))

    plt.bar(y_pos, emotions, align='center', alpha=0.5)
    plt.xticks(y_pos, objects)
    plt.ylabel('percentage')
    plt.title('emotion')

    plt.show()

In [ ]:
img = image.load_img("C:/Users/mike/Pictures/Camera Roll/pablo.png", grayscale=True, target_size=(48, 48))
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)
x /= 255
custom = model.predict(x)
emotion_analysis(custom[0])
x = np.array(x, 'float32')
x = x.reshape([48, 48]);
plt.gray()
plt.imshow(x)
plt.show()